# UEP-0239: Python for Data Analysis and Visualization

---

**A Tufts University Data Lab Tutorial**\
Written by Uku-Kaspar Uustalu

Contact: <uku-kaspar.uustalu@tufts.edu>

Last updated: `2022-03-01`

---

## Importing Packages

We will be using the following Python data analysis and visualization libraries throguhout this tutorial:

- [Pandas](https://pandas.pydata.org/) is the primary data analysis library in Python. It allows for easy analysis and manipulation of tabular data and is usually imported under the alias `pd`.
- [Matplotlib](https://matplotlib.org/) is the most essential data visualization library in Python. Although it consists of many modules, most of the plotting funcionality is contained within the `matplotlib.pyplot` module, which is usually imported under the alias `plt`.
- [Seaborn](https://seaborn.pydata.org/) is an advanced plotting library that is built on top of Matplotlib. It has a simpler interface and allows for the easy creation of beutiful visualizations. Seaborn is usually imported under the alias `sns`.
- [HVPlot](https://hvplot.holoviz.org/) is a high-level plotting interface that integrates seamlessly with Pandas and allows for the easy creation of interactive visualizations. The `hvplot.pandas` module must be imported to allow for seamless integration with Pandas.
- [Plotly](https://plotly.com/) is an alternative interactive visualization library. It consists of many modules, but the `plotly.express` module is the easiest to use as it allows for the creation of whole plots using a single command. The module is usually imported under the alias `px`.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas
import plotly.express as px

---

## Getting Started with Pandas

For the first part of this tutorial, we will be using the following datasets from the `data` folder to investigate the relationship between wealth and health:

- [`gdp.csv`](./data/gdp.csv) – World Bank gross domestic product (GDP) estimates (in USD) for world countries and regions from 1960 until 2020
- [`life-expectancy.csv`](./data/life-exp.csv) – World Bank life expectancy estimates for world countries and regions from 1960 until 2019
- [`m49.csv`](./data/m49.csv) – United Nations [M49](https://en.wikipedia.org/wiki/UN_M49) Standard Country or Area Codes for Statistical Use
- [`population.csv`](./data/population.csv) – World Bank population estimates fror world coutneires and regions from 1960 until 2020

All the datasets are in [RFC 4180 CSV](https://datatracker.ietf.org/doc/html/rfc4180) (comma-separated values) format and the first four rows of the World Bank data files contain metadata with the actual data table starting on row five.

Let us start by reading in the population data. Pandas can easily read CSV datasets via the [`pandas.read_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function. The function reads the contents of the file into a [`pandas.DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) data structure and supports various additional arguments. For example, we can utilize the `skiprows` argument to tell Pandas to skip the frist four rows of the dataset as the data table does not start until fow five.

In [ ]:
population = pd.read_csv('data/population.csv', skiprows=4)

Now the World Bank population dataset is stored in a DataFrame called `population`. Calling the DataFrame by its name will display the first and last five rows of the table by default.

In [ ]:
population

We see that the dataframe appears to have the following columns:

- `Country Name` – English name of the country
- `Country Code` – [ISO 3166-1 alpha-3](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3) country code
- `Indicator Name` – name of the indicator represented by the data
- `Indicator Code` – World Bank code for the indicator
- `1960` ... `2020` – population estimates by year

We also see that the DataFrame has 266 rows and 65 columns. We can double-check this by looking at the value of the `DataFrame.shape` attribute.

In [ ]:
population.shape

The `DataFrame.size` attribue will give us the total number of values in the table (number of columns times number of rows).

In [ ]:
population.size

`DataFrame.columns`can be used to get a list of all the column names and `DataFrame.dtypes` will display the datatype of each column.

In [ ]:
population.columns

In [ ]:
population.dtypes

Note how the first four columns all have the `object` datatype. This denotes either textual data (string) or a mixed datatype (like a list or some other data structure). The population columns are all `float64` denoting floating-point numbers. It might feel odd to store population values as floating-point numbers as population counts are always whole integers. However, in Pandas all numeric data is stored as floating-point numbers by default. This is due to the fact that integer columns in Pandas do not support missing data values ... yet. Currently the default missing data value in Pandas is the `numpy.nan` from NumPy.

We know that the `population` DataFrame stores population values, so the `Indicator Name` and `Indicator Code` columns are redundant. We can drop them from the table using the [`DataFrame.drop()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) method.

In [ ]:
population.drop(columns=['Indicator Name', 'Indicator Code'], inplace=True)

Note how we specified two arguments when calling the [`DataFrame.drop()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) method. Frist we specified a list of columns to drop using the `columns` argument. The [`DataFrame.drop()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) method also supports dropping rows, so that is why the `columns` argument is needed. Then we also specified `inplace` to be `True`. This ensures that the original `population` DataFrame gets modified. Otherwise the method would just return a new DataFrame and keep the `population` dataframe unchanged.

We can validate that the desired columns have been removed by taking a quick peen at the DataFrame via the [`DataFrame.head()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html) method. It displayes the fist five rows of the dataframe by defaut but you can also pass the number of rows desired as an argument.

In [ ]:
population.head()

Knowing that the World Bank GDP dataset follows the extact same format as the World Bank population dataset, we can read it in and drop the `Indicator Name` and `Indicator Code` columns all in one go by chaining together the [`pandas.read_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function and the [`DataFrame.drop()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) method. If we want to include a line break somewhere in the cain, we need to wrap the whole thing in parentheses `()`.

In [ ]:
gdp = (pd.read_csv('data/gdp.csv', skiprows=4)
         .drop(columns=['Indicator Name', 'Indicator Code']))

Note how here we did not specify `inplace=True` when dropping the columns. That is because we want the [`DataFrame.drop()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) method to take the DataFrame generated by [`pandas.read_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) and then output a new DataFrame that we can save into the `gdp` variable. We can take a look at our newly created DataFrame by using the [`DataFrame.head()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html) method again.

In [ ]:
gdp.head()

---

## Long vs Wide Data

GDP on its own is not a good indicator of a coutries wealth as contries with more people tend to have higher GDP. But if we were to normalize GDP by population, then the resulting GDP per capita values can be comared across countries and used as a proxy for wealth. To do so, we must be able to match up the GDP and population values for each unqiue combination of country and year.

The GDP and population tables currently are in wide format – each row represent a unique country and each column represents a unique year with the cell values representing unique population estimates. While this wide format has many advantages and is commonly used in geospatial applications, it does compliate joining various datasets. One option would be to treat both tables and matrices and calculate GDP per capita via by deviding the GDP matric with the population matrix. However, both tables need to have the exact same layout with the same number of countries and years in the same exact order for this to work and the result to be relable. Ensuring this is not a trivial task, so this method would involve a lot of work to produce reliable results.

Alternativeley the two tables could be joined by country. Then we will have an extra-wide table with two sets of year columns – one set of year columns for population and another set of year columns for GDP. Then we would need to create another new column for each year by dividing the corresponding GDP column with the corresponding population column, resulting in another new set of year columns. As you can see, this approach would quickly leed to a vary messy and difficult to manage dataset and would also involve a lot of work, making it far from preferred.

The easiest option for calcluating GDP per capita would involve converting both datasets into a long format, whrere each row represents a single unique observation (estimation). Instead of having countries in rows and years in columns, each row would instead represent a unique country and year combination. This would allow us to easily combine datasets on both country and year, ensuring that the GDP and population values for each country-year combination get matched.

We can use the [`DataFrame.melt()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.melt.html) method to convert wide datafames to long format. We need to specify three argument when using this method:

- `id_vars` – name(s) of the column(s) that define an unique observation in the original wide dataset
- `var_name` – name of the the column in the new long dataset that stores the column names of the original wide dataset
- `value_name` – name of the column int he new long dataset that stores the values of the original wide dataset

Each observation in the original wide dataset represents a unique country defined either by the country name or country code. Let us include both of these as `id_vars` to carry both columns over to the long dataset. The columns of the wide dataset represent years, so that is the name we will pass on to the `var_name` argument. The values of the wide dataset represent population estimates, so that will be the name passed on to the `value_name` argument.

In [ ]:
population_long = population.melt(id_vars=['Country Name', 'Country Code'],
                                  var_name='year',
                                  value_name='population')

In [ ]:
population_long

Now we have a new long population DataFrame called `population_wide`, where each row represent an unique country and year combination. Let us use `DataFrame.dtypes` to confirm the data types of this new talbe.

In [ ]:
population_long.dtypes

Note how the `year` column is of type `object`, meaning that the years are currenty stored as strings. As the years were perviously column names, this make sense. However, as years are actually numbers, they should also be stored as such to allow for easy comarisons and mathematical operations.

In [ ]:
pop_long['year'] = pop_long.year.astype(int)

In [ ]:
pop_long.dtypes

In [ ]:
gdp_long = (gdp.melt(id_vars=['Country Name', 'Country Code'],
                    var_name='year',
                    value_name='gdp')
               .astype({'year': int}))

In [ ]:
gdp_long

In [ ]:
gdp_long.dtypes

---

## Joining Datasets

In [ ]:
data = pop_long.merge(gdp_long,
                      on=['Country Name', 'Country Code', 'year'],
                      how='inner')

In [ ]:
data.head()

In [ ]:
data['gdp_per_capita'] = data.gdp / data.population

In [ ]:
data.head()

In [ ]:
def read_world_bank_data(file_name, value_name):
    return (pd.read_csv(file_name, skiprows=4)
              .drop(columns=['Indicator Name', 'Indicator Code'])
              .melt(id_vars=['Country Name', 'Country Code'],
                    var_name='year',
                    value_name=value_name)
              .astype({'year': int}))

In [ ]:
life_exp = read_world_bank_data(file_name = 'data/life-exp.csv',
                                value_name = 'life_exp')

In [ ]:
life_exp.head()

In [ ]:
data = data.merge(life_exp,
                  on=['Country Name', 'Country Code', 'year'],
                  how='inner')

In [ ]:
data.head()

In [ ]:
data.rename(columns={'Country Name': 'country_name',
                     'Country Code': 'country_code'},
            inplace=True)

In [ ]:
data.head()

In [ ]:
m49 = pd.read_csv('data/m49.csv')

In [ ]:
m49.head()

In [ ]:
regions = m49[['Region Name', 'ISO-alpha3 Code']].copy()

In [ ]:
regions.head()

In [ ]:
regions.rename(columns={'Region Name': 'region_name',
                        'ISO-alpha3 Code': 'country_code'},
               inplace=True)

In [ ]:
regions.head()

In [ ]:
data = data.merge(regions,
                  on='country_code',
                  how='inner')

In [ ]:
data.head()

---

## Boolean Indexing

In [ ]:
usa_data = data[data.country_code == 'USA']

In [ ]:
usa_data.head()

In [ ]:
usa_data.country_name.unique()

In [ ]:
usa_data.country_name.unique()[0]

---

## Creating Visualizations

In [ ]:
plt.plot(usa_data.year, usa_data.population)
plt.show()

In [ ]:
plt.plot('year', 'gdp', data=usa_data)
plt.show()

In [ ]:
usa_data.plot(x='year', y='life_expectancy')
plt.show()

In [ ]:
data[data.country_code == 'USA'].plot(x='year',
                                      y='gdp_per_capita',
                                      color='blue',
                                      label='USA')
data[data.country_code == 'CAN'].plot(x='year',
                                      y='gdp_per_capita',
                                      color='red',
                                      label='Canada',
                                      ax=plt.gca())
plt.ylabel('GDP per capita')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(usa_data.year, usa_data.gdp_per_capita, 'g--', label='GDP per capita')
plt.ylabel('GDP per capita', color='g',)
plt.xlabel('year')
ax2 = ax.twinx()
ax2.plot(usa_data.year, usa_data.life_expectancy, 'mx', label='life expectancy')
plt.ylabel('life expectancy', color='m')
plt.title('USA', size=20)
fig.legend()
plt.show()

---

## Recreating Gapminder

In [ ]:
data2019 = data[data.year == 2019]

In [ ]:
data2019

In [ ]:
plt.hist(data2019.gdp_per_capita)
plt.xlabel('GDP')
plt.show()

In [ ]:
sns.histplot(data2019.life_expectancy, kde=True)
plt.show()

In [ ]:
data2019.plot(x='gdp_per_capita', y='life_expectancy', kind='scatter')
plt.show()

In [ ]:
sns.jointplot(data=data2019,
              x='gdp_per_capita',
              y='life_expectancy',
              kind='kde',
              fill=True)
plt.show()

In [ ]:
plt.scatter(data2019.gdp_per_capita, data2019.life_expectancy)
plt.xscale('log')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.scatter(data2019.gdp_per_capita, data2019.life_expectancy,
            s=data2019.population/data2019.population.max()*5000,
            alpha=0.5)
plt.xscale('log')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
for key, group in data2019.groupby('region_name'):
    plt.scatter(group.gdp_per_capita, group.life_expectancy,
                s=group.population/data2019.population.max()*5000,
                label=key,
                alpha=0.5)
plt.xscale('log')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(data=data2019,
                x='gdp_per_capita',
                y='life_expectancy',
                size='population',
                sizes=(10, 5000),
                hue='region_name',
                alpha=0.5,
                legend=False)
plt.xscale('log')
plt.show()

---

## Interactive Visualizations

In [ ]:
data2019.hvplot.scatter(x='gdp_per_capita',
                        y='life_expectancy',
                        s='population',
                        c='region_name',
                        scale=1/data2019.population.max()*2000000,
                        hover_cols=['country_name', 'country_code'],
                        alpha=0.5,
                        logx=True,
                        width=650,
                        height=500)

In [ ]:
px.scatter(data_frame=data2019.dropna(),
           x='gdp_per_capita',
           y='life_expectancy',
           size='population',
           color='region_name',
           hover_name='country_name',
           hover_data=['country_code'],
           size_max=40,
           opacity=0.5,
           log_x=True,
           width=650,
           height=600)

---

## Working with Timeseries

In [ ]:
mbta = pd.read_csv('data/mbta-gated-entries-2021.csv')

In [ ]:
mbta.head()

In [ ]:
mbta.dtypes

In [ ]:
mbta['time_period'] = mbta.time_period.str.strip('()')

In [ ]:
mbta.head()

In [ ]:
mbta['timestamp'] = pd.to_datetime(mbta.service_date + ' ' + mbta.time_period)

In [ ]:
mbta

In [ ]:
mbta.dtypes

In [ ]:
mbta = mbta[['timestamp', 'station_name', 'route_or_line', 'gated_entries']].copy()

In [ ]:
mbta

In [ ]:
mbta.gated_entries.sum()

In [ ]:
mbta.gated_entries[mbta.timestamp == '2020-02-24'].sum()

In [ ]:
mbta.gated_entries[
    (mbta.timestamp >= '2020-02-01') & (mbta.timestamp < '2020-03-01')].sum()

In [ ]:
mbta.gated_entries[mbta.timestamp.dt.month == 2].sum()

---

## Grouping and Aggregating

In [ ]:
mbta['date'] = mbta.timestamp.dt.date

In [ ]:
mbta_agg = mbta.groupby('date').gated_entries.sum().to_frame().reset_index()

In [ ]:
mbta_agg.head()

In [ ]:
mbta_agg.gated_entries.max()

In [ ]:
mbta_agg.date[mbta_agg.gated_entries == mbta_agg.gated_entries.max()]

In [ ]:
mbta_agg.date[mbta_agg.gated_entries == mbta_agg.gated_entries.max()].values[0]

In [ ]:
(mbta.groupby('station_name')
     .gated_entries.sum()
     .sort_values()
     .to_frame()
     .reset_index())

In [ ]:
(mbta.groupby('route_or_line')
     .gated_entries.sum()
     .sort_values()
     .to_frame()
     .reset_index())

In [ ]:
mbta_bydate = (mbta.groupby(['date', 'station_name', 'route_or_line'])
                   .gated_entries.sum()
                   .to_frame()
                   .reset_index())

In [ ]:
mbta_bydate